In [1]:
import pandas as pd
import numpy as np
import datetime 
from dateutil.relativedelta import relativedelta
from tqdm import tqdm
from scipy import stats

In [2]:
# 데이터 불러오기
weather=pd.read_csv("risk_data/independent_variable/Weather_risk_table_final.csv", encoding='utf-8-sig')
birds=pd.read_csv("risk_data/independent_variable/birds_risk_table_v2.csv", encoding='utf-8-sig')
distance=pd.read_csv("risk_data/independent_variable/distance_risk_final.csv", encoding='utf-8-sig')
dorage=pd.read_csv("risk_data/weight_variable/birds_dorage_risk_table_v1.csv", encoding='utf-8-sig')
density=pd.read_csv("risk_data/weight_variable/density_risk_w_final.csv", encoding='utf-8-sig')

In [3]:
weather.head()

,시도,month,day,avg_temp(C),daily_rain(mm),avg_r_humidity(%),temp_index,rain_index,humidity_index,risk
0,강원영동,1,1,1.2,0.0,33.2,0.0,0.0,0.0,0.0
1,강원영동,1,2,3.2,0.0,34.7,0.0,0.0,0.0,0.0
2,강원영동,1,3,3.4,0.0,35.5,0.0,0.0,0.0,0.0
3,강원영동,1,4,4.2,0.0,39.0,0.0,0.0,0.0,0.0
4,강원영동,1,5,3.2,1.8,43.8,0.0,0.0,0.0,0.0


In [4]:
birds.head()

,month,시도,철새수,발생건수,예측 발생건수,발생건수/예측 발생건수,위험도
0,1,강원도,626130,2,9,0.22,0
1,1,경기도,1242626,55,16,3.44,3
2,1,경상남도,347682,11,6,1.83,1
3,1,경상북도,554884,2,8,0.25,0
4,1,광주광역시,49915,0,2,0.00,0


In [5]:
distance.head()

,month,dist_range,cnt,cumulative,risk
0,1,0,0,0,10
1,1,10,2,2,10
2,1,20,4,6,10
3,1,30,8,14,10
4,1,40,7,21,10


In [6]:
dorage.head()

,시도,시군구,거리,위험분류,risk,cluster
0,서울특별시,종로구,8.85,2,9,0
1,서울특별시,종로구,11.35,3,8,5
2,서울특별시,종로구,13.90,2,9,0
3,서울특별시,종로구,14.68,3,8,5
4,서울특별시,종로구,19.39,1,10,4


In [7]:
density.head()

,지역,밀집도,계급값,계급별가중치,도수분포,가중치점수
0,강원도,205.556164,77.242694,1.01,8.0,1.03
1,경기도,83.456695,154.485389,1.02,2.0,1.02
2,경상남도,113.367620,231.728083,1.03,2.0,1.02
3,경상북도,387.526479,308.970778,1.04,1.0,1.06
4,광주광역시,8.372419,386.213473,1.05,1.0,1.01


In [8]:
# 전국 시군 데이터 가져오기
all_sigun=pd.read_csv("use_data/전국_occur_hpai.csv", encoding='utf-8-sig')

In [9]:
all_sigun.head()

,시도,시군구,발생건수
0,서울특별시,종로구,0
1,서울특별시,중구,0
2,서울특별시,용산구,0
3,서울특별시,성동구,0
4,서울특별시,광진구,2


In [10]:
yeongdong=['강릉시', '동해시', '속초시', '삼척시', '태백시', '고성군', '양양군']
yeongse=['원주시','춘천시','철원군','화천군','양구군','인제군','홍천군','평창군','정선군','영월군','횡성군']

In [11]:
print(set(weather['시도']))

{'경상북도', '충청북도', '강원영서', '전라북도', '강원영동', '서울경기', '경상남도', '전라남도', '제주특별자치도', '충청남도'}


In [12]:
print(set(all_sigun['시도']))

{'광주광역시', '세종특별자치시', '경상북도', '충청북도', '전라북도', '서울특별시', '부산광역시', '경기도', '인천광역시', '울산광역시', '대구광역시', '전라남도', '강원도', '충청남도', '경상남도', '제주특별자치도', '대전광역시'}


In [13]:
# 날씨 시도와 전국 시도 맞추기
w_dic={}
w_dic['인천광역시']='서울경기'
w_dic['대구광역시']='경상북도'
w_dic['부산광역시']='경상남도'
w_dic['울산광역시']='경상남도'
w_dic['서울특별시']='서울경기'
w_dic['대전광역시']='충청남도'
w_dic['경기도']='서울경기'
w_dic['광주광역시']='전라남도'
w_dic['세종특별자치시']='충청남도'
for sido in set(weather['시도']):
    if sido=='강원영동':
        for n in yeongdong:
            w_dic[n]=sido
    elif sido=='강원영서':
        for n in yeongse:
            w_dic[n]=sido
    else:
        w_dic[sido]=sido

In [14]:
print(w_dic)

{'인천광역시': '서울경기', '대구광역시': '경상북도', '부산광역시': '경상남도', '울산광역시': '경상남도', '서울특별시': '서울경기', '대전광역시': '충청남도', '경기도': '서울경기', '광주광역시': '전라남도', '세종특별자치시': '충청남도', '경상북도': '경상북도', '충청북도': '충청북도', '원주시': '강원영서', '춘천시': '강원영서', '철원군': '강원영서', '화천군': '강원영서', '양구군': '강원영서', '인제군': '강원영서', '홍천군': '강원영서', '평창군': '강원영서', '정선군': '강원영서', '영월군': '강원영서', '횡성군': '강원영서', '전라북도': '전라북도', '강릉시': '강원영동', '동해시': '강원영동', '속초시': '강원영동', '삼척시': '강원영동', '태백시': '강원영동', '고성군': '강원영동', '양양군': '강원영동', '서울경기': '서울경기', '경상남도': '경상남도', '전라남도': '전라남도', '제주특별자치도': '제주특별자치도', '충청남도': '충청남도'}


In [15]:
set(distance['month'])

{1, 2, 3, 4, 5, 6, 7, 9, 10, 11, 12}

In [16]:
distance.head()

,month,dist_range,cnt,cumulative,risk
0,1,0,0,0,10
1,1,10,2,2,10
2,1,20,4,6,10
3,1,30,8,14,10
4,1,40,7,21,10


In [17]:
dorage.head()

,시도,시군구,거리,위험분류,risk,cluster
0,서울특별시,종로구,8.85,2,9,0
1,서울특별시,종로구,11.35,3,8,5
2,서울특별시,종로구,13.90,2,9,0
3,서울특별시,종로구,14.68,3,8,5
4,서울특별시,종로구,19.39,1,10,4


In [18]:
all_sigun.head()

,시도,시군구,발생건수
0,서울특별시,종로구,0
1,서울특별시,중구,0
2,서울특별시,용산구,0
3,서울특별시,성동구,0
4,서울특별시,광진구,2


In [19]:
# 시군구별 지역별 철새 도래지의 따른 가중치 부여
dorage_risk=[]
for n1, group in dorage.groupby(['시도','시군구']):
    dorage_risk.append(np.round(group['risk'].mean(),2))  # 103개의 철새도래지의 위험도를 시군별로 평균값을 구함.
print("일부 데이터:",dorage_risk[:20])
print("평균:",np.mean(dorage_risk))
print("최대값:",max(dorage_risk))
print("최솟값:",min(dorage_risk))
print("25%:",np.quantile(dorage_risk,.25))
print("75%:",np.quantile(dorage_risk,.75))
print("25% 이하 개수:",sum(list(map(lambda x : x<=5.51,dorage_risk))))
print("75% 이상 개수:",sum(list(map(lambda x : x>=6.53,dorage_risk))))
print("사잇값 개수:",sum(list(map(lambda x : x<6.53 and x>5.51,dorage_risk))))
print("분할값:",np.round((6.53-5.51)/9,5))

일부 데이터: [4.54, 4.18, 4.59, 4.61, 4.27, 4.94, 4.44, 5.75, 6.4, 4.82, 5.17, 5.11, 5.56, 5.02, 5.69, 6.06, 5.08, 6.25, 5.91, 6.2]
평균: 6.006375545851529
최대값: 7.14
최솟값: 2.1
25%: 5.51
75%: 6.53
25% 이하 개수: 58
75% 이상 개수: 58
사잇값 개수: 113
분할값: 0.11333


In [20]:
# 철새 도래지 가중치 dic 만들기
# 25% 이하는 0, 75% 이상은 0.1, 중간 값은 0.01에서 0.09까지 9분할 
dorage_dic={}

for n1, group in dorage.groupby(['시도','시군구']):
    risk_temp=np.round(group['risk'].mean(),2)
    if risk_temp<=5.51:
        weights=0
    elif risk_temp>=6.53:
        weights=0.1
    else: 
        weights=0
        left=5.51
        right=left+(6.53-5.51)/9
        for i in range(1,11):
            if risk_temp>left and risk_temp<right:
                weights+=np.round(i/100,2)
                break
            left=right
            right=left+(6.53-5.51)/9
    dorage_dic[n1]=weights

In [21]:
dorage_dic[('강원도','춘천시')]

0.01

In [22]:
density.head()

,지역,밀집도,계급값,계급별가중치,도수분포,가중치점수
0,강원도,205.556164,77.242694,1.01,8.0,1.03
1,경기도,83.456695,154.485389,1.02,2.0,1.02
2,경상남도,113.367620,231.728083,1.03,2.0,1.02
3,경상북도,387.526479,308.970778,1.04,1.0,1.06
4,광주광역시,8.372419,386.213473,1.05,1.0,1.01


In [23]:
# 시도별 밀집도 가중치 dic 만들기
dic_density={}
for i in range(len(density)):
    dic_density[density.loc[i,'지역']]=density.loc[i,'가중치점수']

In [24]:
# 전체 데이터 SET 만들기
cols=['month','day','시도','시군구','철새수','최초발생거리','평균온도','일강수량','상대습도','위험도']
total_df=pd.DataFrame(columns=cols)
total_index_df=pd.DataFrame(columns=cols)
total=[]
total_index=[]
for month in range(1,13):
    start_date='2020-'+str(month)+'-01'
    start_date=datetime.datetime.strptime(start_date, '%Y-%m-%d')
    end_date=start_date+relativedelta(months=1)-datetime.timedelta(days=1)
    for day in tqdm(range(1,end_date.day+1)):
        for i in range(len(all_sigun)):
            q3=distance.query("month==@month").reset_index(drop=True)
            for j in range(len(q3)):
                temp={}
                index_temp={}
                temp['month']=month
                temp['day']=day
                sido=all_sigun.loc[i,'시도']
                temp['시도']=sido
                sigun=all_sigun.loc[i,'시군구']
                temp['시군구']=sigun
                index_temp['month']=month
                index_temp['day']=day
                index_temp['시도']=sido
                index_temp['시군구']=sigun

                q1=birds.query("month==@month and 시도==@sido").reset_index(drop=True)
                if len(q1)==0:
                    temp['철새수']=0
                    index_temp['철새수']=0
                else:
                    temp['철새수']=q1.loc[0,'철새수']
                    index_temp['철새수']=q1.loc[0,'위험도']

                if temp['시도']=='강원도':
                    r_sigun=w_dic[sigun]
                    q2=weather.query("month==@month and day==@day and 시도==@r_sigun").reset_index(drop=True)
                else:
                    r_sido=w_dic[sido]
                    q2=weather.query("month==@month and day==@day and 시도==@r_sido").reset_index(drop=True)
                temp['평균온도']=q2.loc[0,'avg_temp(C)']
                temp['일강수량']=q2.loc[0,'daily_rain(mm)']
                temp['상대습도']=q2.loc[0,'avg_r_humidity(%)']
                index_temp['평균온도']=q2.loc[0,'temp_index']
                index_temp['일강수량']=q2.loc[0,'rain_index']
                index_temp['상대습도']=q2.loc[0,'humidity_index']
                
                # 지역간 거리 부여
                temp['최초발생거리']=q3.loc[j,'dist_range']
                index_temp['최초발생거리']=q3.loc[j,'risk']
                
                # 위험도 가중치 부여 (밀집도+철새도래지거리)
                index_temp['위험도']=dic_density[sido]+dorage_dic[(sido,sigun)]
                total.append(temp)
                total_index.append(index_temp)

100%|██████████████████████████████████████████████████████████████████████████████████| 31/31 [17:45<00:00, 34.38s/it]


In [25]:
total_df=total_df.append(total)
total_index_df=total_index_df.append(total_index)

In [45]:
total_df.to_csv("risk_data/Train_data_set(risk_x).csv", encoding='utf-8-sig', index=False)

In [46]:
total_index_df.to_csv("risk_data/Train_risk_data_set(risk_x).csv", encoding='utf-8-sig', index=False)

In [47]:
total_df=pd.read_csv("risk_data/Train_data_set(risk_x).csv", encoding='utf-8-sig')

In [48]:
total_index_df=pd.read_csv("risk_data/Train_risk_data_set(risk_x).csv", encoding='utf-8-sig')

In [42]:
total_index_df.query("시군구=='종로구'").reset_index(drop=True)

,month,day,시도,시군구,철새수,최초발생거리,평균온도,일강수량,상대습도,철새도래지거리,도래지분류,위험도
0,1,1,서울특별시,종로구,0,10,10.0,10.0,10.0,NaN,NaN,1.1
1,1,1,서울특별시,종로구,0,10,10.0,10.0,10.0,NaN,NaN,1.1
2,1,1,서울특별시,종로구,0,10,10.0,10.0,10.0,NaN,NaN,1.1
3,1,1,서울특별시,종로구,0,10,10.0,10.0,10.0,NaN,NaN,1.1
4,1,1,서울특별시,종로구,0,10,10.0,10.0,10.0,NaN,NaN,1.1
...,...,...,...,...,...,...,...,...,...,...,...,...
10799,12,31,서울특별시,종로구,0,0,10.0,0.0,10.0,NaN,NaN,1.1
10800,12,31,서울특별시,종로구,0,0,10.0,0.0,10.0,NaN,NaN,1.1
10801,12,31,서울특별시,종로구,0,0,10.0,0.0,10.0,NaN,NaN,1.1
10802,12,31,서울특별시,종로구,0,0,10.0,0.0,10.0,NaN,NaN,1.1


In [31]:
total_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2474116 entries, 0 to 2474115
Data columns (total 12 columns):
 #   Column   Dtype  
---  ------   -----  
 0   month    int64  
 1   day      int64  
 2   시도       object 
 3   시군구      object 
 4   철새수      int64  
 5   최초발생거리   int64  
 6   평균온도     float64
 7   일강수량     float64
 8   상대습도     float64
 9   철새도래지거리  float64
 10  도래지분류    float64
 11  위험도      float64
dtypes: float64(6), int64(4), object(2)
memory usage: 226.5+ MB


In [32]:
# 시도, 시군구 범주화 하기
encoded_value, encoded_class = pd.factorize(total_df['시도'])

In [33]:
encoded_value2, encoded_class2 = pd.factorize(total_df['시군구'])

In [34]:
total_df.head()

,month,day,시도,시군구,철새수,최초발생거리,평균온도,일강수량,상대습도,철새도래지거리,도래지분류,위험도
0,1,1,서울특별시,종로구,126690,0,-1.8,0.0,64.2,NaN,NaN,NaN
1,1,1,서울특별시,종로구,126690,10,-1.8,0.0,64.2,NaN,NaN,NaN
2,1,1,서울특별시,종로구,126690,20,-1.8,0.0,64.2,NaN,NaN,NaN
3,1,1,서울특별시,종로구,126690,30,-1.8,0.0,64.2,NaN,NaN,NaN
4,1,1,서울특별시,종로구,126690,40,-1.8,0.0,64.2,NaN,NaN,NaN


In [35]:
sido_sigun=total_df[['시도','시군구']]
sido_sigun['encoded_시도'] = encoded_value
sido_sigun['encoded_시군구'] = encoded_value2
total_df['시도'] = encoded_value
total_df['시군구'] = encoded_value2

C:\Users\user\AppData\Local\Temp/ipykernel_9292/133994541.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sido_sigun['encoded_시도'] = encoded_value
C:\Users\user\AppData\Local\Temp/ipykernel_9292/133994541.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sido_sigun['encoded_시군구'] = encoded_value2


In [36]:
sido_sigun=sido_sigun.drop_duplicates().reset_index(drop=True)

In [37]:
sido_sigun.head()

,시도,시군구,encoded_시도,encoded_시군구
0,서울특별시,종로구,0,0
1,서울특별시,중구,0,1
2,서울특별시,용산구,0,2
3,서울특별시,성동구,0,3
4,서울특별시,광진구,0,4


In [38]:
sido_sigun.to_csv("risk_data/sido_sigun_index.csv", index=False, encoding='utf-8-sig')

In [39]:
sido_sigun.head()

,시도,시군구,encoded_시도,encoded_시군구
0,서울특별시,종로구,0,0
1,서울특별시,중구,0,1
2,서울특별시,용산구,0,2
3,서울특별시,성동구,0,3
4,서울특별시,광진구,0,4


In [40]:
total_df.head()

,month,day,시도,시군구,철새수,최초발생거리,평균온도,일강수량,상대습도,철새도래지거리,도래지분류,위험도
0,1,1,0,0,126690,0,-1.8,0.0,64.2,NaN,NaN,NaN
1,1,1,0,0,126690,10,-1.8,0.0,64.2,NaN,NaN,NaN
2,1,1,0,0,126690,20,-1.8,0.0,64.2,NaN,NaN,NaN
3,1,1,0,0,126690,30,-1.8,0.0,64.2,NaN,NaN,NaN
4,1,1,0,0,126690,40,-1.8,0.0,64.2,NaN,NaN,NaN


In [52]:
# 다중공선성 체크
# 10 이상인 변수도 있고, 엄격하게는 5이상의 변수도 많이 포함되어 있음
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor

X = total_df[list(total_df.columns[:-1])]

vif_info = pd.DataFrame()
vif_info['VIF'] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
vif_info['Column'] = X.columns
vif_info.sort_values('VIF', ascending=False)

,VIF,Column
2,44.595635,시도
3,39.385011,시군구
8,20.186932,상대습도
6,6.113155,평균온도
0,4.936486,month
1,4.152259,day
5,3.631421,최초발생거리
4,2.088440,철새수
7,1.761452,일강수량


In [53]:
# 시도, 시군구 상관관계 큼 (다중공선성 10 이상)
# 시도+시군구 번호를 합치기 (다중공선성 제거)
total_df['시도']=total_df['시도']+total_df['시군구']
total_df.drop(['시군구'], axis=1, inplace=True)
total_df=total_df.rename(columns={'시도':'시군구index'})

In [54]:
total_df.head()

,month,day,시군구index,철새수,최초발생거리,평균온도,일강수량,상대습도,위험도
0,1,1,0,126690,0,-1.8,0.0,64.2,NaN
1,1,1,0,126690,10,-1.8,0.0,64.2,NaN
2,1,1,0,126690,20,-1.8,0.0,64.2,NaN
3,1,1,0,126690,30,-1.8,0.0,64.2,NaN
4,1,1,0,126690,40,-1.8,0.0,64.2,NaN


In [55]:
# 시군 인덱스 데이터 표도 합치기
sido_sigun['index']=sido_sigun['encoded_시도']+sido_sigun['encoded_시군구']
sido_sigun.to_csv("risk_data/sido_sigun_index.csv", index=False, encoding='utf-8-sig')

In [56]:
# 다중공선성 재확인하기
X = total_df[list(total_df.columns[:-1])]

vif_info = pd.DataFrame()
vif_info['VIF'] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
vif_info['Column'] = X.columns
vif_info.sort_values('VIF', ascending=False)

,VIF,Column
7,20.010753,상대습도
5,6.059743,평균온도
0,4.921274,month
1,4.152078,day
4,3.631421,최초발생거리
2,3.447256,시군구index
3,1.949115,철새수
6,1.759529,일강수량


In [57]:
# 상대습도 데이터는 다중공선성이 10이상이며, 독립변수로 역할이 부족할 것이라고 판단 -> 데이터 삭제
total_df.drop(['상대습도'], axis=1, inplace=True)
total_index_df.drop(['상대습도'], axis=1, inplace=True)

In [58]:
# 최종 위험도 부여 (철새수: 0.3, 최초발생거리:0.5, 평균온도:0.14, 일강수량:0.06)
total_index_df['위험도']*=(total_index_df['철새수']*0.3+total_index_df['최초발생거리']*0.5+total_index_df['평균온도']*0.14+total_index_df['일강수량']*0.06)
total_index_df['위험도']=total_index_df['위험도'].map(lambda x : np.round(x,1))

In [59]:
total_index_df.head()

,month,day,시도,시군구,철새수,최초발생거리,평균온도,일강수량,위험도
0,1,1,서울특별시,종로구,0,10,10.0,10.0,7.7
1,1,1,서울특별시,종로구,0,10,10.0,10.0,7.7
2,1,1,서울특별시,종로구,0,10,10.0,10.0,7.7
3,1,1,서울특별시,종로구,0,10,10.0,10.0,7.7
4,1,1,서울특별시,종로구,0,10,10.0,10.0,7.7


In [60]:
total_df['위험도']=total_index_df['위험도']

In [61]:
total_df.head()

,month,day,시군구index,철새수,최초발생거리,평균온도,일강수량,위험도
0,1,1,0,126690,0,-1.8,0.0,7.7
1,1,1,0,126690,10,-1.8,0.0,7.7
2,1,1,0,126690,20,-1.8,0.0,7.7
3,1,1,0,126690,30,-1.8,0.0,7.7
4,1,1,0,126690,40,-1.8,0.0,7.7


In [65]:
total_df.query("시군구index==10").reset_index(drop=True).head()

,month,day,시군구index,철새수,최초발생거리,평균온도,일강수량,위험도
0,1,1,10,126690,0,-1.8,0.0,7.6
1,1,1,10,126690,10,-1.8,0.0,7.6
2,1,1,10,126690,20,-1.8,0.0,7.6
3,1,1,10,126690,30,-1.8,0.0,7.6
4,1,1,10,126690,40,-1.8,0.0,7.6


In [63]:
total_df.isnull().sum()

month       0
day         0
시군구index    0
철새수         0
최초발생거리      0
평균온도        0
일강수량        0
위험도         0
dtype: int64

In [67]:
max(total_df['위험도'])

11.6

In [68]:
min(total_df['위험도'])

0.0

In [64]:
total_df.to_csv("./risk_data/Train_data_set.csv", encoding='utf-8-sig', index=False)

In [66]:
total_index_df.to_csv("./risk_data/Train_risk_data_set.csv", encoding='utf-8-sig', index=False)